## Parallel Version Solved Using Atomic

In [7]:
#import the necessary libraries
import os
import cv2
import numpy as np
import time
from joblib import Parallel, delayed
from multiprocessing import Manager, Value, Lock
import multiprocessing

In [8]:
# Folder containing the images
input_folder = 'cars'  # Replace with the path to your folder
output_folder = 'cars_filtered'  # Replace with the path to save filtered images

# Creating the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [9]:
# Function to process rows (done in parallel)
def process_row(row_index, image, filter_kernel, filter_size, height, width, atomic_rows,lock,cores):

    output_row = np.zeros(width, dtype=np.float32)
    for j in range(width):
        weighted_sum = 0.0
        for k in range(filter_size):
            for l in range(filter_size):
                row = row_index + k - filter_size // 2
                col = j + l - filter_size // 2
                if 0 <= row < height and 0 <= col < width:
                    weighted_sum += image[row, col] * filter_kernel[k, l]
        output_row[j] = weighted_sum

    # Atomic section: Updating processed row count
    with lock:
        atomic_rows.value += 1

    return output_row

In [10]:
# Main function with parallel processing of rows using joblib
def parallel_process(image, atomic_rows,lock,cores):

    # Preparing for parallel processing
    height, width = image.shape

    filter_kernel = np.array([[0.1, 0.2, 0.1],
                              [0.2, 0.1, 0.2],
                              [0.1, 0.2, 0.1]], dtype=np.float32)
    filter_size = filter_kernel.shape[0]

    # Parallelizing the row processing
    results = Parallel(n_jobs=cores)(
        delayed(process_row)(i, image, filter_kernel, filter_size, height, width, atomic_rows,lock,cores)
        for i in range(height)
    )

    # Reconstructing the output image from the processed rows
    output_image = np.array(results, dtype=np.float32)
    return output_image

In [11]:
# Function to process and save each image (Done in parallel)
def process_image(image_path, atomic_images, atomic_rows,lock,cores):

    # Reading the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"Could not open or find the image: {image_path}")
        return
    
    # Applying Gaussian Blur
    blurred_image = cv2.GaussianBlur(image, (3, 3), 0)

    # Computing the elevation map using Sobel operator for edge detection
    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)

    # Combining the gradients to obtain the magnitude of the gradient
    magnitude_gradient = cv2.magnitude(sobel_x, sobel_y)

    # Normalizing the image to a range between 0 and 1 (floating-point operations)
    image = magnitude_gradient.astype(np.float32) / 255.0

    # Applying parallel custom filter operation
    output_image = parallel_process(image, atomic_rows,lock,cores)

    # Clipping the output to stay in the range [0, 1]
    output_image = np.clip(output_image, 0.0, 1.0)

    # Converting the processed image back to 8-bit (0-255)
    processed_image = (output_image * 255).astype(np.uint8)

    # Saving the processed image
    output_image_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_image_path, processed_image)

    # Atomic section: Updating processed image count
    with lock:
        atomic_images.value += 1

In [12]:
# List of image paths to process
image_paths = [
    os.path.join(input_folder, filename)
    for filename in os.listdir(input_folder)
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp'))
]

core_counts=[1,2,3,4,6,8,10,12,14,16]
for cores in core_counts:
    # Manager provides a way to share data safely between processes
    with Manager() as manager:
    # Atomic values for shared counters
        lock = manager.Lock()
        atomic_images = manager.Value('i', 0)  # Number of processed images
        atomic_rows = manager.Value('i', 0)    # Number of processed rows

        print(f"\nTesting with {cores} cores:")
        start_time = time.time()

        # Parallelizing the image processing using joblib
        Parallel(n_jobs=cores)(
            delayed(process_image)(image_path, atomic_images, atomic_rows,lock, cores)
            for image_path in image_paths
        )   

        # Printing
        print(f"Total time taken by the execution of this code for all images using Joblib: {time.time() - start_time:.2f}s")
        print(f"Processed Images (atomic): {atomic_images.value}")
        print(f"Processed Rows (atomic): {atomic_rows.value}")


Testing with 1 cores:
Total time taken by the execution of this code for all images using Joblib: 429.92s
Processed Images (atomic): 200
Processed Rows (atomic): 96000

Testing with 2 cores:
Total time taken by the execution of this code for all images using Joblib: 196.24s
Processed Images (atomic): 200
Processed Rows (atomic): 96000

Testing with 3 cores:
Total time taken by the execution of this code for all images using Joblib: 140.75s
Processed Images (atomic): 200
Processed Rows (atomic): 96000

Testing with 4 cores:
Total time taken by the execution of this code for all images using Joblib: 137.47s
Processed Images (atomic): 200
Processed Rows (atomic): 96000

Testing with 6 cores:
Total time taken by the execution of this code for all images using Joblib: 171.17s
Processed Images (atomic): 200
Processed Rows (atomic): 96000

Testing with 8 cores:
Total time taken by the execution of this code for all images using Joblib: 188.15s
Processed Images (atomic): 200
Processed Rows (a